In [2]:
import polars as pl
from pprint import pprint

In [71]:
def fix_str_to_int(data:pl.DataFrame, fix_list:list, type:any=pl.Int64):
    conversion_dict = {column:pl.Int64 for column in fix_list}
    data = data.with_columns(pl.col(fix_list).str.split(","))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip()))
    data = data.with_columns(pl.col(fix_list).list.eval(pl.element().cast(pl.Int64, strict=False)))

    for column in fix_list:
        data = data.with_columns(pl.when(pl.col(column).list.eval(pl.element().is_null()).list.any()).then(None).otherwise(pl.col(column)).alias("bools"))
        data = data.select(replace(data, data.get_column_index(column), "bools")).rename({"bools":column})
    
    # data = data.with_columns(pl.col(fix_list).str.strip().str.replace("","0")).cast(conversion_dict)

    return data


# Abych nebyl nařčen z plagiátorství, zdroj: https://stackoverflow.com/questions/75954280/how-to-change-the-position-of-a-single-column-in-python-polars-library
def replace(df, new_position, col_name):
    neworder=df.columns
    neworder.pop(new_position)
    neworder.remove(col_name)
    neworder.insert(new_position,col_name)
    return neworder


In [72]:
rozvrh_by_kat = pl.read_csv("source_tables/getRozvrhByKatedra-2023-03-15-17-47.csv").drop("semestr").unique().rename({"predmet" : "zkratka"})
rozvrh_by_kat = fix_str_to_int(rozvrh_by_kat, ["ucitIdno.ucitel", "vsichniUciteleUcitIdno"])
#print(rozvrh_by_kat.head())
predmety_by_kat = pl.read_csv("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.csv").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])
predmety_by_kat = fix_str_to_int(predmety_by_kat, ["garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "hodZaSemKombForma"])
predmety_by_kat.head(10)
#predmety_by_kat = pl.read_excel("source_tables/getPredmetyByKatedraFullInfo-2023-03-15-17-40.xlsx", sheet_id=1, engine="openpyxl").drop("semestr").unique().drop(["pocetStudentu", "aSkut", "bSkut", "cSkut"])

C:\Users\Matej\AppData\Local\Temp\ipykernel_16204\1838726611.py:4: DeprecationWarning: `strip` is deprecated. It has been renamed to `strip_chars`.
  data = data.with_columns(pl.col(fix_list).list.eval(pl.element().str.strip()))


katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""APSIM""",2023,"""System Simulat…","""System Simulat…","""A""","""N""",1,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""""","""""",null,"""""","""""",null,"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…","[251, 3521]","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",0,"""HOD/TYD""",1,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Angličtina""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""DI3""",2023,"""Didaktika info…","""Didaktika info…","""A""","""N""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…","[1165, 2776]","""""","""""","""""","""""","""KI/DI2, KI/KDI…","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""1. tvořivá prá…",""" 1. Učebnice i…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""
"""KI""","""DPG""",2023,"""Didaktika prog…","""Didaktika prog…","""A""","""N""",2,"""NE""","""""","""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…",[1165],"""""","""""",null,"""""","""""",null,"""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…",[1165],"""""","""""","""""","""""","""""","""""","""""","""""","""""","""Pracovní činno…","""Rozbor produkt…","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Student využij…","""Zápočet""","""NE""","""Kombinovaná""","""- Aktivní účas…","""1. Cíle, prost…","""""","""Absolvent kurz…","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""EICT""",2023,"""Information an…","""Information an…","""A""","""A""",8,"""NE""","""""","""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""The course wil…","""Zkouška""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Angličtina""","""""","""A""","""N""","""A""","""2/3 per week""","""""","""""","""N""","""N""","""""",0,"""""","""""","""A""",[0],"""""","""""",""""""
"""KI""","""KANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'do

In [138]:
# qr = ~pl.all().is_duplicated()
# rozvrh_by_kat = rozvrh_by_kat.filter(qr)
# predmety_by_kat = predmety_by_kat.filter(qr)


In [27]:
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str
"""KI""","""AIT""",2023,"""Architektura a…","""Architektura a…","""A""","""N""",3,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…","""2776""","""""","""""","""""","""""","""""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…","""2776""","""""","""""","""""","""""","""KI/ZPP, KI/ZPS…","""""","""KI/IVP""","""'Velte, A. T.,…","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Kurz je zaměře…","""Zkouška""","""NE""","""Kombinovaná""","""Zápočet: labor…","""1. IT a její ž…","""Základny počít…","""""","""""","""""","""Čeština""","""Tento kurz vzn…","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""""","""""",""""""
"""KI""","""ANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…","""5232""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…","""5232""","""'doc. RNDr. Fr…","""'doc. RNDr. Fr…","""5232""","""""","""""","""""","""""","""""","""""","""""","""KI/ZEL""","""""","""""","""'Tichý, M. Ele…","""""","""""","""""","""N""",1,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Kurz je zaměře…","""Zápočet""","""NE""","""Kombinovaná""","""Ústní obhajoba…","""1. Pasivní ele…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""0""","""""","""""",""""""
"""KI""","""APC""",2023,"""Architektura p…","""Architektura p…","""A""","""N""",3,"""NE""","""""","""'doc. RNDr. Mg…","""'doc. RNDr. Mg…","""2239""","""'Ing. Pavel Ku…","""'Ing. Pavel Ku…","""2317, 2239""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""KI/DHW, KI/PPT…","""'Horák J. Hard…","""""","""""","""""","""N""",2,"""HOD/TYD""",0,"""HOD/TYD""",0,"""HOD/TYD""","""Tento předmět …","""Zkouška""","""NE""","""Kombinovaná""","""Zkouška je kom…","""1) Vývoj PC, g…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""""","""""","""""",""""""
"""KI""","""BTE""",2023,"""Bezpečnostní t…","""Bezpečnostní t…","""A""","""N""",4,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…","""2776""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""KI/ZKB, KI/KZK…","""""","""KI/ASK""","""'Sanders C. An…","""""","""""","""""","""A""",1,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet: labor…","""1. Infrastrukt…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Bc.""","""A""","""0""","""""",""

In [75]:
rozvrh_by_kat.head(10)

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,str,list[i64],str,str,str,str,str,str,str,i64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,list[i64],str,str,str,i64,str,str,str,str
491621,"""GUI Programmin…","""KI""","""EGUI""","""""","""650""",[650],"""Petr""","""Kubera""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",6.13,30,4,2,"""Přednáška""","""Př""","""A""","""Čtvrtek""","""Čt""","""""",2,3,2,"""08:00""","""09:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""""","""UJEP""","""4.4.2024""","""""","""""","""R""","[650, 6973]","""RNDr. Petr Kub…","""'RNDr. Petr Ku…","""Kubera, Beráne…",491621,"""""","""""","""RSKVARA""",""""""
490642,"""Algoritmizace …","""KI""","""APR2""","""""","""6973""",[6973],"""Pavel""","""Beránek""","""Ing. Mgr.""","""""","""A""","""A""","""100""",2023,"""CP""",6.14,31,38,38,"""Cvičení""","""Cv""","""A""","""Čtvrtek""","""Čt""","""""",6,7,2,"""12:00""","""13:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""1.API Bc,1.MvF…","""UJEP""","""4.4.2024""","""""","""""","""R""",[6973],"""Ing. Mgr. Pave…","""'Ing. Mgr. Pav…","""Beránek""",490642,"""""","""""","""RSKVARA""",""""""
490643,"""Základy zpraco…","""KI""","""ZZD""","""""","""2220""",[2220],"""Jiří""","""Škvor""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""MF""",2.28,190,41,39,"""Seminář""","""Se""","""A""","""Pátek""","""Pá""","""""",7,7,1,"""13:00""","""13:50""",8,20,"""Každý""","""K""","""31861""","""N""","""N""","""Automaticky vy…","""1.AJ INF,1.API…","""UJEP""","""5.4.2024""","""""","""""","""R""","[2220, 449, 6259]","""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvor, Barilla…",490643,"""""","""""","""RSKVARA""",""""""
490644,"""Základy zpraco…","""KI""","""ZZD""","""""","""2220""",[2220],"""Jiří""","""Škvor""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""MF""",2.28,190,41,40,"""Seminář""","""Se""","""A""","""Pátek""","""Pá""","""""",7,7,1,"""13:00""","""13:50""",8,20,"""Každý""","""K""","""31862""","""N""","""N""","""Automaticky vy…","""1.AJ INF,1.API…","""UJEP""","""5.4.2024""","""""","""""","""R""","[2220, 449, 6259]","""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvor, Barilla…",490644,"""""","""""","""RSKVARA""",""""""
486986,"""Číslicové syst…","""KI""","""CSY""","""""","""3416""",[3416],"""Sergii""","""Babichev""","""prof.""","""CSc., DSc.""","""A""","""A""","""100""",2023,"""CP""",7.07,12,12,3,"""Přednáška""","""Př""","""A""","""Pátek""","""Pá""","""""",8,8,1,"""14:00""","""14:50""",8,20,"""Každý""","""K""","""""","""N""","""N""","""Automaticky vy…","""2.API Bc,3.API…","""UJEP""","""5.4.2024""","""""","""""","""R""",[3416],"""prof. Sergii B…","""'prof. Sergii …","""Babichev""",486986,"""""","""""","""RSKVARA""",""""""
491150,"""Semestrální pr…","""KI""","""SP2""","""""","""2220""",[2220],"""Jiří""","""Škvor""","""RNDr.""","""Ph.D.""","""A""","""A""","""100""",2023,"""CP""",6.01,2,1,1,"""Seminář""","""Se""","""A""","""Pátek""","""Pá""","""""",10,12,3,"""16:00""","""18:50""",8,20,"""Sudý""","""S""","""31922""","""N""","""N""","""Automaticky vy…","""1.API N""","""UJEP""","""5.4.2024""","""""","""""","""R""","[2220, 6259]","""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvor, Škvára""",491150,"""""","""""","""RSKVARA""",""""""
486988,"""Programování h…","""KI""","""PHW2""","""""",""

In [17]:
set(rozvrh_by_kat.columns) & set(predmety_by_kat.columns)

{'katedra', 'nazev', 'rok', 'zkratka'}

In [113]:
rozvrh_by_kat.null_count()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
4,0,0,0,0,0,42,0,0,0,0,0,0,0,0,0,146,146,4,0,0,0,0,0,0,0,153,153,148,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,42,0,0,0,0,0,0,0,0


In [114]:
predmety_by_kat.null_count()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,45,0,0,308,0,0,291,0,0,295,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,231,0,0,0


In [144]:
# vybírám pouze předměty, které mají rozvrhovou akci

In [77]:
predmety_s_akci = predmety_by_kat.join(other=rozvrh_by_kat, on="zkratka", how="inner").select(predmety_by_kat.columns).unique().sort("zkratka")
str(len(predmety_s_akci)) + "   " + str(len(predmety_by_kat)) # ...what does this do
predmety_s_akci.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""0033""",2023,"""Optimalizace v…","""Optimalizace v…","""A""","""A""",2,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Lagová M., Ja…","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Náplní výběrov…","""Zápočet""","""NE""","""Ústní""","""Vstupní požada…","""1) Úvod do mat…","""""","""""","""""","""""","""Čeština""","""vk 1528""","""A""","""N""","""N""","""""","""1""","""1""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0121""",2023,"""CCNA 1: Introd…","""CCNA 1: Introd…","""A""","""A""",1,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",1,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cílem předmětu…","""""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0126""",2023,"""CCNA 2: Routin…","""CCNA 2: Routin…","""A""","""A""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cílem předmětu…","""CCNA 1""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0127""",2023,"""CCNA 3: Scalin…","""CCNA 3: Scalin…","""A""","""A""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",2,"""HOD/TYD""",0,"""HOD/TYD""","""Cílem předmětu…","""Zápočet""","""NE""","""Kombinovaná""","""Splnění závěre…","""Cíle předmětu:…","""CCNA 2""","""""","""""","""""","""Čeština, Angli…","""vk""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""0190""",2023,"""CCNA Security""","""CCNA Security""","""A""","""A""",2,"""NE""","""""","""'

# Vyučující a garanti

In [78]:
jednotek_prednasek = pl.col("jednotekPrednasek")
jednotek_cviceni = pl.col("jednotekCviceni")
jednotek_seminare = pl.col("jednotekSeminare")
garant = pl.col("garanti")
cvicici = pl.col("cvicici")
prednasejici = pl.col("prednasejici")
seminarici = pl.col("seminarici")

### Každý předmět má právě jednoho garanta

> Zde jsou předměty, které nemají garanta:

In [82]:
zkratky = predmety_s_akci.filter(pl.col("garantiUcitIdno").is_null()).select(["zkratka", "rok", "nazev", "nazevDlouhy", "garanti", "garantiSPodily"]).filter(
    # Není SZ
    # pl.col("zkratka").str.starts_with("SZ").is_not()
    True
)
print(f"\n{len(zkratky)} Předmětů nemá zadaného garanta")
zkratky.write_excel("results/bez_garanta.xlsx")
zkratky
# print(list(zkratky), sep=", ")


17 Předmětů nemá zadaného garanta


zkratka,rok,nazev,nazevDlouhy,garanti,garantiSPodily
str,i64,str,str,str,str
"""BOP""",2023,"""Odborná praxe""","""Odborná praxe""","""""",""""""
"""BP2O""",2023,"""Bakalářská prá…","""Bakalářská prá…","""""",""""""
"""BPP""",2023,"""Bakalářská prá…","""Bakalářská prá…","""""",""""""
"""DPP""",2023,"""Diplomová prác…","""Diplomová prác…","""""",""""""
"""EDAV""",2023,"""Data Analysis …","""Data Analysis …","""""",""""""
…,…,…,…,…,…
"""SZZIV""",2023,"""Informatika pr…","""Informatika pr…","""""",""""""
"""SZZPP""",2023,"""Aplikovaná inf…","""Aplikovaná inf…","""""",""""""
"""SZZTD""",2023,"""Teoretické zák…","""Teoretické zák…","""""",""""""


> Předměty podezřelé z více garantů:

In [87]:
# predmety_s_akci.filter( > pl.lit())
# Nefunguje, arr.lengths údajně nic není
vice_garantu = predmety_s_akci.with_columns(pl.col("garantiUcitIdno").list.len().alias("pocet garantu")
    ).select(
        ["garantiUcitIdno", "garanti", "zkratka", "pocet garantu"]
    ).filter(pl.col("pocet garantu") > 1)
vice_garantu.write_excel("results/vice_garantu.xlsx")
vice_garantu.head(10)

garantiUcitIdno,garanti,zkratka,pocet garantu
list[i64],str,str,u32


### Každý předmět má alespoň jednoho vyučujícího v kategoriích dle parametrů předmětu

např. pokud má předmět rozsah 2 h přednášek a 2 h cvičení, pak musí mít alespoň jednoho přednášejícího a alespoň jednoho cvičícího a zároveň nesmí mít vyučujícího v kategorii vede seminář.

In [88]:
ma_prednasku = jednotek_prednasek != 0
nema_prednasejiciho = pl.col("prednasejiciUcitIdno").is_null()

chybi_prednasejici = predmety_s_akci.filter(ma_prednasku & nema_prednasejiciho)
chybi_prednasejici.write_excel("results/chybi_prednasejici.xlsx")
chybi_prednasejici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KVICT""",2023,"""Využití ICT ve…","""Využití ICT ve…","""N""","""A""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",4,"""HOD/SEM""",0,"""HOD/SEM""",2,"""HOD/SEM""","""1. Mezipředmět…","""Zápočet""","""NE""","""Kombinovaná""","""- tvořivá prác…","""1. Mezipředmět…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""


In [90]:
ma_cviceni = jednotek_cviceni != 0
nema_cviciciho = pl.col("cviciciUcitIdno").is_null()

chybi_cvicici = predmety_s_akci.filter(ma_cviceni & nema_cviciciho)
chybi_cvicici.write_excel("results/chybi_cvicici.xlsx")
chybi_cvicici.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""BOP""",2023,"""Odborná praxe""","""Odborná praxe""","""A""","""N""",24,"""NE""","""""","""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",12,"""TYD/SEM""",0,"""HOD/TYD""","""Odborná praxe …","""Zápočet""","""NE""","""Kombinovaná""","""Zápočet je udě…","""Odborná praxe …","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""


In [91]:
ma_seminare = jednotek_seminare != 0
nema_seminariciho = pl.col("seminariciUcitIdno").is_null()

chybi_seminarici = predmety_s_akci.filter(ma_seminare & nema_seminariciho)
chybi_seminarici.write_excel("results/chybi_seminarici.xlsx")
chybi_seminarici

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""KAFJ""",2023,"""Teorie automat…","""Teorie automat…","""A""","""N""",5,"""NE""","""""","""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…",[449],"""'doc. Ing. Mgr…","""'doc. Ing. Mgr…","[449, 2220]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""'Chytil M. Aut…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""V tomto předmě…","""Zkouška""","""ANO""","""Kombinovaná""","""Požadavky pro …","""1. Konečné aut…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""","""0"""
"""KI""","""KAPR1""",2023,"""Algoritmizace …","""Algoritmizace …","""A""","""N""",4,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","[6973, 650, … 6259]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/KAPR2, KI/K…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Úvodní kurz al…","""Zápočet""","""NE""","""Kombinovaná""","""Podmínkou získ…","""1. Základní te…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KAPR2""",2023,"""Algoritmizace …","""Algoritmizace …","""N""","""A""",5,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","[650, 835, 6259]","""""","""""",null,"""""","""""","""""","""""","""KI/KAPR1""","""""","""KI/KGUI, KI/KP…","""'https://githu…","""""","""""","""""","""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Druhá část úvo…","""Zkouška""","""ANO""","""Kombinovaná""","""Podmínkou získ…","""1. Implementac…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""N""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""KCSY""",2023,"""Číslicové syst…","""Číslicové syst…","""N""","""A""",4,"""NE""","""""","""'prof. Sergii …","""'prof. Sergii …",[3416],"""'prof. Sergii …","""'prof. Sergii …",[3416],"""'prof. Sergii …","""'prof. Sergii …","[3416, 1780]","""""","""""",null,"""""","""""","""""","""""","""""","""""","""KI/PCSY""","""'Gook M. Hardw…","""""","""""","""""","""N""",4,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Student se v r…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: semes…","""1. Syntéza kom…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""

> Zde jsou předměty, ve kterých učitel vede seminář:

In [35]:
#prednasejici_je_seminarici = prednasejici == seminarici
#cvicici_je_seminarici = cvicici == seminarici

#konflikt_vyucujicich = predmety_s_akci.filter(ma_seminare & ma_prednasku & (prednasejici_je_seminarici | cvicici_je_seminarici))
#konflikt_vyucujicich.write_excel("results/konflikt_vyucujicich.xlsx")
#konflikt_vyucujicich.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,null,str,str,str,str,str,str,str,str,str,str,str,str,null,null,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,null,null,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,null,null,str
"""KI""","""KDIS""",2023,"""Dependabilita …","""Dependabilita …","""A""","""A""",4,"""NE""",null,"""'doc. RNDr. Mg…","""'doc. RNDr. Mg…","""2239""","""'doc. RNDr. Mg…","""'doc. RNDr. Mg…","""2239""","""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","""6973, 2239""","""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","""6973, 2239""",null,null,null,null,"""KI/KAPR1""",null,null,"""'Laprie, J.C.,…",null,null,null,"""N""",6,"""HOD/SEM""",4,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz uvádí do …","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: semin…","""1. Úvod do pro…",null,null,null,null,"""Čeština, Angli…","""Výuka v anglic…","""A""","""N""","""N""",null,null,null,"""N""","""A""",null,0,"""Bc.""","""Bc.""","""A""",null,null,null,null
"""KI""","""KGUI""",2023,"""Programování p…","""Programování p…","""N""","""A""",6,"""NE""",null,"""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","""650""","""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","""6973, 650""","""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","""6973, 650""","""'Ing. Mgr. Pav…","""'Ing. Mgr. Pav…","""6973, 650""",null,null,null,null,"""KI/KAPR2, KI/K…",null,null,"""'Sells Ch. C# …",null,null,null,"""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je zaměře…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet je udě…","""1. Model událo…",null,null,null,null,"""Čeština""",null,"""A""","""N""","""N""",null,null,null,"""N""","""A""",null,0,null,"""Bc.""","""A""",null,null,null,null
"""KI""","""KOPR""",2023,"""Optimální rozh…","""Optimální rozh…","""A""","""A""",5,"""NE""",null,"""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","""650""","""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","""650, 9114""","""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","""650, 9114""","""'RNDr. Petr Ku…","""'RNDr. Petr Ku…","""650, 9114""",null,null,null,null,null,null,null,"""'Lagová, M. Me…",null,null,null,"""N""",6,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Kurz je zaměře…","""Zkouška""","""ANO""","""Kombinovaná""","""Podmínkou získ…","""1. Úlohy lineá…",null,null,null,null,"""Čeština""",null,"""A""","""N""","""N""",null,null,null,"""N""","""A""",null,0,null,"""Bc.""","""A""","""0""",null,null,null
"""KI""","""KPAU""",2023,"""Programovateln…","""Programovateln…","""A""","""N""",4,"""NE""",null,"""'Ing. Petr Hab…","""'Ing. Petr Hab…","""1780""","""'Ing. Petr Hab…","""'Ing. Petr Hab…","""1780""","""'Ing. Petr Hab…","""'Ing. Petr Hab…","""1780""","""'Ing. Petr Hab…","""'Ing. Petr Hab…","""1780""",null,null,null,null,null,null,"""KI/KSCAD""","""'Šmejkal, L., …",null,null,null,"""A""",4,"""HOD/SEM""",6,"""HOD/SEM""",2,"""HOD/SEM""","""Cílem předmětu…","""Zkouška""","""ANO""","""Kombinovaná""","""Zápočet: semes…","""1. Úvod do pro…",null,null,null,null,"""Čeština""",null,"""A""","""N""","""N""",n

In [92]:
print(f"{len(chybi_prednasejici)} předmětů nemá přednášejícího")
print(f"{len(chybi_cvicici)} předmětů nemá cvičícího")
print(f"{len(chybi_seminarici)} předmětů nemá seminářícího")
# print(f"U {len(konflikt_vyucujicich)} předmětů je seminářící také učitel")

1 předmětů nemá přednášejícího
1 předmětů nemá cvičícího
29 předmětů nemá seminářícího


### Garant by měl být někde mezi vyučujícími. Pokud má předmět přednášku, měl by být přednášející.

> V těchto předmětech garant vůbec neučí:

In [95]:
filter_query = (garant != cvicici) & (garant != seminarici) & (garant != prednasejici) & (ma_prednasku | ma_cviceni | ma_seminare)
predmety_kde_garant_neuci = predmety_s_akci.filter(filter_query)
predmety_kde_garant_neuci.write_excel("results/predmety_kde_garant_neuci.xlsx")
predmety_kde_garant_neuci.__len__()

55

> v těchto předmětech je přednáška, ale nedělá jí garant:

In [97]:
garant_neni_prednasejici = (jednotek_prednasek != 0) & (garant != prednasejici)
garant_neprednasi = predmety_s_akci.filter(garant_neni_prednasejici)
garant_neprednasi.write_excel("results/garant_neprednasi.xlsx")
garant_neprednasi.__len__()

28

## Kontrola proti rozvrhovaným akcím

### Vyučující u rozvrhových akcí musí být v příslušných kategoriích u vyučujících

#### Osekám dataframy

In [156]:
rozvrh_by_kat.head()
# predmety_by_kat.head()

roakIdno,nazev,katedra,zkratka,statut,ucitIdno,ucitIdno.ucitel,jmeno.ucitel,prijmeni.ucitel,titulPred.ucitel,titulZa.ucitel,platnost.ucitel,zamestnanec.ucitel,podilNaVyuce.ucitel,rok,budova,mistnost,kapacitaMistnosti,planObsazeni,obsazeni,typAkce,typAkceZkr,platnost,den,denZkr,vyucJazyk,hodinaOd,hodinaDo,pocetVyucHodin,hodinaSkutOd,hodinaSkutDo,tydenOd,tydenDo,tyden,tydenZkr,grupIdno,jeNadrazena,maNadrazenou,kontakt,krouzky,casovaRada,datum,datumOd,datumDo,druhAkce,vsichniUciteleUcitIdno,vsichniUciteleJmenaTituly,vsichniUciteleJmenaTitulySPodily,vsichniUcitelePrijmeni,referencedIdno,poznamkaRozvrhare,nekonaSe,owner,zakazaneAkce
i64,str,str,str,str,i64,i64,str,str,str,str,str,str,i64,i64,str,f64,i64,i64,i64,str,str,str,str,str,str,i64,i64,i64,str,str,i64,i64,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str
446685,"""Počítačové sít…","""KI""","""PSI""",null,2776,2776,"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""",100,2022,"""CP""",7.07,12,12,3,"""Cvičení""","""Cv""","""A""","""Pondělí""","""Po""",null,4,5,2,"""10:00""","""11:50""",40,1,"""Každý""","""K""",null,"""N""","""N""","""Automaticky vy…","""3.IS""","""UJEP""",null,"""03.10.2022""","""02.01.2023""","""R""","""2776""","""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",446685,null,null,"""PETRUSPRF""",null
446500,"""Základy počíta…","""KI""","""ZPS""",null,2776,2776,"""Jan""","""Krejčí""","""RNDr.""","""Ph.D.""","""A""","""A""",100,2022,"""MF""",1.54,315,180,170,"""Přednáška""","""Př""","""A""","""Pondělí""","""Po""",null,7,8,2,"""13:00""","""14:50""",40,1,"""Každý""","""K""",null,"""N""","""N""","""Automaticky vy…","""1.API Bc""","""UJEP""",null,"""03.10.2022""","""02.01.2023""","""R""","""2776""","""RNDr. Jan Krej…","""'RNDr. Jan Kre…","""Krejčí""",446500,null,null,"""PETRUSPRF""",null
446519,"""Didaktika prog…","""KI""","""DPG""",null,1165,1165,"""Kamil""","""Balín""","""Mgr.""",null,"""A""","""A""",100,2022,"""CP""",7.06,24,20,14,"""Seminář""","""Se""","""A""","""Pondělí""","""Po""",null,8,9,2,"""14:00""","""15:50""",40,1,"""Každý""","""K""",null,"""N""","""N""","""Automaticky vy…","""2.API Bc,3.IS,…","""UJEP""",null,"""03.10.2022""","""02.01.2023""","""R""","""1165""","""Mgr. Kamil Bal…","""'Mgr. Kamil Ba…","""Balín""",446519,null,"""17.10.2022: Od…","""PETRUSPRF""",null
446723,"""Informační a k…","""KI""","""IKT""",null,835,835,"""Květuše""","""Sýkorová""","""Mgr.""",null,"""A""","""A""",100,2022,"""CP""",-1.21,31,30,30,"""Seminář""","""Se""","""A""","""Úterý""","""Út""",null,10,11,2,"""16:00""","""17:50""",40,1,"""Každý""","""K""",null,"""N""","""N""","""Automaticky vy…","""1.MvFVS,1.NANO…","""UJEP""",null,"""04.10.2022""","""03.01.2023""","""R""","""835""","""Mgr. Květuše S…","""'Mgr. Květuše …","""Sýkorová""",446723,null,"""25.10.2022: Od…","""PETRUSPRF""",null
451651,"""Semestrální pr…","""KI""","""KSP3""",null,6259,6259,"""Jiří""","""Škvára""","""RNDr.""","""Ph.D.""","""A""","""A""",100,2022,"""CP""",6.04,2,1,1,"""Seminář""","""Se""","""A""","""Úterý""","""Út""",null,12,13,2,"""18:00""","""19:50""",1,1,"""Jiný""","""J""",28039,"""N""","""N""","""Automaticky vy…","""2.KAPI N""","""UJEP""",null,"""03.01.2023""","""03.01.2023""","""R""","""6259""","""RNDr. Jiří Škv…","""'RNDr. Jiří Šk…","""Škvára""",451651,null,null,"""PETRUSPRF""",null


In [106]:
maly_rozvrh = rozvrh_by_kat.select(["zkratka", "vsichniUciteleUcitIdno", "typAkceZkr", "rok"])
maly_rozvrh = maly_rozvrh.with_columns(pl.col("vsichniUciteleUcitIdno")).explode("vsichniUciteleUcitIdno").rename({"vsichniUciteleUcitIdno": "idno"})
maly_rozvrh.head()

zkratka,idno,typAkceZkr,rok
str,i64,str,i64
"""EGUI""",650,"""Př""",2023
"""EGUI""",6973,"""Př""",2023
"""APR2""",6973,"""Cv""",2023
"""ZZD""",2220,"""Se""",2023
"""ZZD""",449,"""Se""",2023


In [158]:
# predmety_by_kat

In [98]:
from pprint import pprint
# pprint(predmety_by_kat.columns,compact=True)
predmety_by_kat.head()

katedra,zkratka,rok,nazev,nazevDlouhy,vyukaZS,vyukaLS,kreditu,viceZapis,minObsazeni,garanti,garantiSPodily,garantiUcitIdno,prednasejici,prednasejiciSPodily,prednasejiciUcitIdno,cvicici,cviciciSPodily,cviciciUcitIdno,seminarici,seminariciSPodily,seminariciUcitIdno,schvalujiciUznani,schvalujiciUznaniUcitIdno,examinatori,examinatoriUcitIdno,podminujiciPredmety,vylucujiciPredmety,podminujePredmety,literatura,nahrazPredmety,metodyVyucovaci,metodyHodnotici,akreditovan,jednotekPrednasek,jednotkaPrednasky,jednotekCviceni,jednotkaCviceni,jednotekSeminare,jednotkaSeminare,anotace,typZkousky,maZapocetPredZk,formaZkousky,pozadavky,prehledLatky,predpoklady,ziskaneZpusobilosti,casovaNarocnost,predmetUrl,vyucovaciJazyky,poznamka,ectsZobrazit,ectsAkreditace,ectsNabizetUPrijezdu,poznamkaVerejna,skupinaAkreditace,skupinaAkreditaceKey,zarazenDoPrezencnihoStudia,zarazenDoKombinovanehoStudia,studijniOpory,praxePocetDnu,urovenNastavena,urovenVypoctena,automatickyUznavatZppZk,hodZaSemKombForma,aMax,bMax,cMax
str,str,i64,str,str,str,str,i64,str,str,str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,list[i64],str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,list[i64],str,str,str
"""KI""","""APSIM""",2023,"""System Simulat…","""System Simulat…","""A""","""N""",1,"""NE""","""""","""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…",[251],"""""","""""",null,"""""","""""",null,"""'Mgr. Jiří Fiš…","""'Mgr. Jiří Fiš…","[251, 3521]","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""A""",0,"""HOD/TYD""",0,"""HOD/TYD""",1,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Angličtina""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",[0],"""""","""""",""""""
"""KI""","""DI3""",2023,"""Didaktika info…","""Didaktika info…","""A""","""N""",2,"""NE""","""""","""'RNDr. Jan Kre…","""'RNDr. Jan Kre…",[2776],"""""","""""",null,"""""","""""",null,"""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…","[1165, 2776]","""""","""""","""""","""""","""KI/DI2, KI/KDI…","""""","""""","""""","""""","""""","""""","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""""","""Zápočet""","""NE""","""Kombinovaná""","""1. tvořivá prá…",""" 1. Učebnice i…","""""","""""","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""N""","""""",0,"""""","""Mgr.""","""A""",[0],"""""","""""",""""""
"""KI""","""DPG""",2023,"""Didaktika prog…","""Didaktika prog…","""A""","""N""",2,"""NE""","""""","""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…",[1165],"""""","""""",null,"""""","""""",null,"""'Mgr. Kamil Ba…","""'Mgr. Kamil Ba…",[1165],"""""","""""","""""","""""","""""","""""","""""","""""","""""","""Pracovní činno…","""Rozbor produkt…","""N""",0,"""HOD/TYD""",0,"""HOD/TYD""",2,"""HOD/TYD""","""Student využij…","""Zápočet""","""NE""","""Kombinovaná""","""- Aktivní účas…","""1. Cíle, prost…","""""","""Absolvent kurz…","""""","""""","""Čeština""","""""","""A""","""N""","""N""","""""","""""","""""","""A""","""A""","""""",0,"""""","""Bc.""","""A""",null,"""""","""""",""""""
"""KI""","""EICT""",2023,"""Information an…","""Information an…","""A""","""A""",8,"""NE""","""""","""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""'Ing. Pavel Ku…","""'Ing. Pavel Ku…",[2317],"""""","""""",null,"""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""""","""N""",2,"""HOD/TYD""",3,"""HOD/TYD""",0,"""HOD/TYD""","""The course wil…","""Zkouška""","""NE""","""Kombinovaná""","""""","""""","""""","""""","""""","""""","""Angličtina""","""""","""A""","""N""","""A""","""2/3 per week""","""""","""""","""N""","""N""","""""",0,"""""","""""","""A""",[0],"""""","""""",""""""
"""KI""","""KANE""",2023,"""Analogová elek…","""Analogová elek…","""A""","""N""",2,"""NE""","""""","""'doc. RNDr. Fr…","""'do

In [100]:
male_predmety = predmety_by_kat.select(pl.col(["zkratka", "katedra", "rok", "nazev", "garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno"]).where(pl.col("zkratka").is_unique()))
#male_predmety = male_predmety.with_columns(pl.col(["prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno", "garantiUcitIdno"]).str.split(", "))
male_predmety.head()

C:\Users\Matej\AppData\Local\Temp\ipykernel_16204\3219921638.py:1: DeprecationWarning: `where` is deprecated. Use `filter` instead.
  male_predmety = predmety_by_kat.select(pl.col(["zkratka", "katedra", "rok", "nazev", "garantiUcitIdno", "prednasejiciUcitIdno", "cviciciUcitIdno", "seminariciUcitIdno"]).where(pl.col("zkratka").is_unique()))


zkratka,katedra,rok,nazev,garantiUcitIdno,prednasejiciUcitIdno,cviciciUcitIdno,seminariciUcitIdno
str,str,i64,str,list[i64],list[i64],list[i64],list[i64]
"""APSIM""","""KI""",2023,"""System Simulat…",[251],null,null,"[251, 3521]"
"""DI3""","""KI""",2023,"""Didaktika info…",[2776],null,null,"[1165, 2776]"
"""DPG""","""KI""",2023,"""Didaktika prog…",[1165],null,null,[1165]
"""EICT""","""KI""",2023,"""Information an…",[2317],[2317],[2317],null
"""KANE""","""KI""",2023,"""Analogová elek…",[5232],[5232],[5232],null


In [107]:
# přednášející nemá přednášku:
filtrovani_prednasejici = male_predmety.select("nazev", "zkratka", "prednasejiciUcitIdno").explode("prednasejiciUcitIdno").filter(
    pl.col("prednasejiciUcitIdno"
).is_not_null()).with_columns(
    pl.col("prednasejiciUcitIdno").alias("idno")
)
joined_prednasejici = filtrovani_prednasejici.join(maly_rozvrh, "idno", "left")
prednasejici_bez_prednasek = joined_prednasejici.filter(pl.col("typAkceZkr").is_null())
prednasejici_bez_prednasek.select("nazev", "zkratka", "prednasejiciUcitIdno").sort("prednasejiciUcitIdno").write_excel("results/prednasejici_bez_prednasek.xlsx")

In [108]:
# cvičící nemá cvičení:
filtrovani_cvicici = male_predmety.select("nazev", "zkratka", "cviciciUcitIdno").explode("cviciciUcitIdno").filter(
    pl.col("cviciciUcitIdno"
).is_not_null()).with_columns(
    pl.col("cviciciUcitIdno").alias("idno")
)
joined_cvicici = filtrovani_cvicici.join(maly_rozvrh, "idno", "left")
cvicici_bez_cviceni = joined_cvicici.filter(pl.col("typAkceZkr").is_null())
cvicici_bez_cviceni.select("nazev", "zkratka", "cviciciUcitIdno").sort("cviciciUcitIdno").write_excel("results/cvicici_bez_cviceni.xlsx")

In [109]:
# přednášející nemá přednášku:
filtrovani_seminarici = male_predmety.select("nazev", "zkratka", "seminariciUcitIdno").explode("seminariciUcitIdno").filter(
    pl.col("seminariciUcitIdno"
).is_not_null()).with_columns(
    pl.col("seminariciUcitIdno").alias("idno")
)
joined_seminarici = filtrovani_seminarici.join(maly_rozvrh, "idno", "left")
seminarici_bez_seminare = joined_seminarici.filter(pl.col("typAkceZkr").is_null())
seminarici_bez_seminare.select("nazev", "zkratka", "seminariciUcitIdno").sort("seminariciUcitIdno").write_excel("results/seminarici_bez_seminare.xlsx")

### warning, když je mezi vyučujícími někdo bez odpovídajícího typu rozvrhové akce

Pro každého učitele vypíšu předměty, u kterých má mít nějakou akci.

Takže budu mít dataframe, kde bude učitel, zkratka, a jeden sloupeček za každý typ akce, kde učitel bude muset mít aspoň 1 narozvrhovaný typ akce s daným předmětem, kde je hodnota `True`

In [112]:
# vypíšu typy akcí
list(maly_rozvrh.select(pl.col("typAkceZkr").unique()))

[shape: (5,)
 Series: 'typAkceZkr' [str]
 [
 	"Se"
 	"Zkouška"
 	"Cv"
 	"Př"
 	"Zápočet"
 ]]

### garant předmětu by měl být někde mezi vyučujícími rozvrhových akcí a pokud má předmět přednášku, měl by být i mezi přednášejícími v rámci rozvrhových akcí

### Změny proti akreditacím.

### Porovnat revize předmětů

tj. parametry předmětů pro aktuální akademický rok a pro příští.

> jakou to má mít podobu?